<a href="https://colab.research.google.com/github/spirosChv/neuro208/blob/main/practicals/Practical_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spiking neuronal models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# @title Make plots nicer  -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.rcParams.update(mystyle())

---
## About units

- Capacitance ($C_m$): $\text{Farad}$ or $\text{F}$, $1F = 1 C/V = 1 s/\Omega$, in SI $kg^{-1} \cdot m^{-2} \cdot s^4 \cdot A^2$
- Leak Conductance ($g_L$) in $\text{Siemens}$ or $S$ or $\Omega^{-1}$ or $\text{mho}$, in SI $1S = kg^{-1} \cdot m^{-2} \cdot s^3 \cdot A^2$
- Membrane time constant $\tau_m$ in $\text{seconds}$ or $s$

Thus,

\begin{equation}
\tau_m = C_m \cdot R_m = \frac{C_m}{g_L}
\end{equation}

is given in $seconds$. However, in Neurosciense we use time constants expressed in $miliseconds$ or $ms$.


Let's assume that $C_m = 2 pF$ and $g_L = 0.1 nS$, then $\tau_m = 20ms$.

---
## Example 1: Leaky Integrate-and-Fire (LIF) model

\begin{equation}
C_m \frac{dV}{dt} = -g_L (V-E_L) + I_e
\end{equation}

where $C_m$ is the membrane capacitance, $g_L$ and $E_L$ are the leak conductance and reversal potential, reespectively, and $I_e$ is an input current.

Spike mechanism:

\begin{equation}
\text{if } V \ge V_{th} \text{ then } V → V_r
\end{equation}

In [ ]:
# Simulation parameters
T = 2000  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

In [ ]:
# Model Parameters
Cm = 300  # membrane capacitance in pF
gL = 15  # leaky conductance in nS
EL = -70  # leaky reversal potential in mV

Vth = -54  # threshold in mV
Vr = -80  # reset in mV

In [ ]:
# Create the exteral current
Ie = np.zeros((len(t_vector), ))

t1 = 100  # ms
t2 = t1 + 1000  # ms

n1 = int(t1/dt)
n2 = int(t2/dt)

Ie[n1:n2 + 1] = 300  # current amplitude in pA

plt.figure(figsize=(8, 6))
plt.plot(t_vector, Ie)
plt.xlabel('time (ms)')
plt.ylabel('current (pA)')
plt.show()

In [ ]:
# Simulation
V = np.zeros((len(t_vector), ))  # Array to store the voltage
V[0] = EL  # initialize voltage at rest
tspike = []  # empty list to store spiketimes.

# Write the Euler method
for n in range(steps):
  ##################################
  # YOUR CODE GOES HERE
  ##################################
  deriv = ...
  V[n+1] = ...

  # Spike mechanism
  if V[n+1] > Vth:  # if V is greater than the threshold
    tspike.append(n*dt)  # record the spike time
    ##################################
    # YOUR CODE GOES HERE
    ##################################
    V[n+1] = ...  # reset the voltage

In [ ]:
# Print the spiketimes
for t in tspike:
  print(f"Spike at: {np.round(t, 1)} ms")

print(f"\nNumber of spikes: {len(tspike)}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(t_vector, V)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.show()

In [ ]:
# Draw nicer spikes
for t in tspike:
  n = int(t/dt)
  V[n] = 20

plt.figure(figsize=(8, 6))
plt.plot(t_vector, V)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.show()

Let's add a refractory period, $t_{ref}$, during which the neuron cannot spike!

In [ ]:
tref = 20  # refracrtory period in ms

In [ ]:
V_LIF = np.zeros((len(t_vector), ))
V_LIF[0] = EL
tspike = [-1]  # add a negative value for programming convinience.
flag = True

for n in range(steps):
  deriv = (-gL*(V_LIF[n]-EL) + Ie[n])/Cm
  V_LIF[n+1] = V_LIF[n] + dt*deriv

  if V_LIF[n+1] > Vth and flag:
    tspike.append(n*dt)
    V_LIF[n+1] = Vr
    flag = False

  if not flag:
    V_LIF[n + 1] = Vr

  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Print the spiketimes
tspike_LIF = tspike[1:]  # remove the 'fake' spiketime
for t in tspike_LIF:
  print(f"Spike at: {np.round(t, 1)} ms")

print(f"\nNumber of spikes: {len(tspike_LIF)}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(t_vector, V_LIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('LIF model')
plt.show()

In [ ]:
# Draw nicer spikes
for t in tspike_LIF:
  n = int(t/dt)
  V_LIF[n] = 20

plt.figure(figsize=(8, 6))
plt.plot(t_vector, V_LIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('LIF model')
plt.show()

---
## Example 2: Quadratic Integrate-and-Fire (QIF) model

\begin{equation}
C_m \frac{dV}{dt} = -g_L \frac{(V-E_L)(V_{th} - V)}{V_{th}-E_L} + I_e
\end{equation}


where $C_m$ is the membrane capacitance, $g_L$ and $E_L$ are the leak conductance and reversal potential, reespectively. $V_{th}$ is the spike threshold, and $I_e$ is an input current. Once the membrane potential crosses $V_{th}$, it diverges to infinity in finite time. In numerical simulation the integration is stopped if the membrane potential hits an arbitrary threshold $V_{peak}$ (much larger than $V_{th}$) at which the membrane potential is reset to a value $V_r$.


Spike mechanism:

\begin{equation}
\text{if } V \ge V_{peak} \text{ then } V → V_r
\end{equation}

In [ ]:
# Simulation parameters
T = 2000  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

# Model Parameters
C = 300  # membrane capacitance in pF
gL = 15  # leaky conductance in nS
EL = -70  # leaky reversal potential in mV

Vth = -54  # threshold in mV
Vr = -80  # reset in mV
Vpeak = 20 # peak potential (detection limit) in mV

In [ ]:
V_QIF = np.zeros((len(t_vector), ))
V_QIF[0] = EL
tspike = [-1]  # add a negative value for programming convinience.
flag = True

for n in range(steps):
  ##################################
  # YOUR CODE GOES HERE
  ##################################
  deriv = ...
  V_QIF[n+1] = ...

  if V_QIF[n+1] > Vpeak and flag:
    tspike.append(n*dt)
    V_QIF[n+1] = Vr
    flag = False

  if not flag:
    V_QIF[n+1] = Vr

  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Print the spiketimes
tspike_QIF = tspike[1:]  # remove the 'fake' spiketime
for t in tspike_QIF:
  print(f"Spike at: {np.round(t, 1)} ms")

print(f"\nNumber of spikes: {len(tspike_QIF)}")

In [ ]:
for t in tspike_QIF:
  n = int(t/dt)
  V_QIF[n] = 20

plt.figure(figsize=(8, 6))
plt.plot(t_vector, V_QIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('QIF model')
plt.show()

---
## Example 3: Exponential Integrate-and-Fire (EIF) model

\begin{equation}
C_m \frac{dV}{dt} = -g_L (V-E_L) + g_L \Delta_T \text{exp} \left( \frac{V-V_T}{\Delta_T} \right) + I_e
\end{equation}

where $C_m$ is the membrane capacitance, $g_L$ and $E_L$ are the leak conductance and reversal potential, reespectively. $\Delta_T$ controls the sharpness of action potential initiation (typical values $ \sim 1 mV$ for pyramidal neurons), $V_T$ denotes the intrinsic membrane potential threshold. Once the membrane potential crosses $V_{T}$, it diverges to infinity in finite time. In numerical simulation the integration is stopped if the membrane potential hits an arbitrary threshold $V_{peak}$ (much larger than $V_{T}$) at which the membrane potential is reset to a value $V_r$.

In the limit $\Delta_T → 0$, the EIF becomes LIF.

Spike mechanism:

\begin{equation}
\text{if } V \ge V_{peak} \text{ then } V → V_r
\end{equation}

In [ ]:
# Simulation parameters
T = 2000  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

# Model Parameters
C = 300  # membrane capacitance in pF
gL = 15  # leaky conductance in nS
EL = -70  # leaky reversal potential in mV

VT = -54  # threshold in mV
DeltaT = 2  # AP sharpness (slope factor) in mV
Vr = -80  # reset in mV
Vpeak = 20 # peak potential (detection limit) in mV

In [ ]:
V_EIF = np.zeros((len(t_vector), ))
V_EIF[0] = EL
tspike = [-1]  # add a negative value for programming convinience.
flag = True

for n in range(steps):
  ##################################
  # YOUR CODE GOES HERE
  ##################################
  deriv = ...
  V_EIF[n+1] = ...

  if V_EIF[n+1] > Vpeak and flag:
    tspike.append(n*dt)
    V_EIF[n+1] = Vr
    flag = False

  if not flag:
    V_EIF[n+1] = Vr

  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Print the spiketimes
tspike_EIF = tspike[1:]  # remove the 'fake' spiketime
for t in tspike_EIF:
  print(f"Spike at: {np.round(t, 1)} ms")

print(f"\nNumber of spikes: {len(tspike_EIF)}")

In [ ]:
for t in tspike_EIF:
  n = int(t/dt)
  V_EIF[n] = 20

plt.figure(figsize=(8, 6))
plt.plot(t_vector, V_EIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('EIF model')
plt.show()

In [ ]:
# Plot all together
t1 = 400
t2 = 470

n1 = int(t1/dt)
n2 = int(t2/dt)

plt.figure(figsize=(8, 6))
plt.plot(t_vector[n1:n2+1], V_LIF[n1:n2+1], label='LIF')
plt.plot(t_vector[n1:n2+1], V_QIF[n1:n2+1], label='QIF')
plt.plot(t_vector[n1:n2+1], V_EIF[n1:n2+1], label='EIF')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.legend()
plt.show()

---
## Example 4: Adaptive Exponential Integrate-and-Fire neuronal (aEIF) model

\begin{align}
C_m \frac{dV}{dt} &= -g_L (V-E_L) + g_L \Delta_T \text{exp} \left( \frac{V-V_T}{\Delta_T} \right) - w + I_e \\
\tau_w \frac{dw}{dt} &= \alpha (V-E_L) - w
\end{align}

- where $C_m$ is the membrane capacitance, $g_L$ and $E_L$ are the leak conductance and reversal potential, reespectively.

- $\Delta_T$ controls the sharpness of action potential initiation (typical values $ \sim 1 mV$ for pyramidal neurons), $V_T$ denotes the intrinsic membrane potential threshold. Once the membrane potential crosses $V_{T}$, it diverges to infinity in finite time. In numerical simulation the integration is stopped if the membrane potential hits an arbitrary threshold $V_{peak}$ (much larger than $V_{T}$) at which the membrane potential is reset to a value $V_r$.

- $\tau_w$ is the time constant and $\alpha$ represents the level of **subthreshold adaptation**. At each firing time, the variable $w$ is increased by anamount $b$, which accounts for **spike-triggered adaptation**.

Spike mechanism:

\begin{equation}
\text{if } V \ge V_{peak} \text{ then } V → V_r, w → w + b
\end{equation}

In [ ]:
# Simulation parameters
T = 2000  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

# Model Parameters
C = 300  # membrane capacitance in pF
gL = 15  # leaky conductance in nS
EL = -70  # leaky reversal potential in mV

VT = -54  # threshold in mV
DeltaT = 2  # AP sharpness (slope factor) in mV
Vr = -80  # reset in mV
Vpeak = 20 # peak potential (detection limit) in mV

# Adaptation
tau_w = 144 # adaptation time constant in ms
alpha = 4  # subthreshold adaptation in nS
b = 80.5 # spike-triggered adaptation in pA

In [ ]:
V_aEIF = np.zeros((len(t_vector), ))
V_aEIF[0] = EL
w = np.zeros((len(t_vector), ))
w[0] = 0
tspike = [-1]  # add a negative value for programming convinience.
flag = True

for n in range(steps):
  ##################################
  # YOUR CODE GOES HERE
  ##################################
  deriv = ...
  V_aEIF[n+1] = ...

  deriv_w = ...
  w[n+1] = ...

  if V_aEIF[n+1] > Vpeak and flag:
    tspike.append(n*dt)
    V_aEIF[n+1] = Vr
    w[n+1] += b
    flag = False

  if not flag:
    V_aEIF[n+1] = Vr

  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Print the spiketimes
tspike_aEIF = tspike[1:]  # remove the 'fake' spiketime
for t in tspike_aEIF:
  print(f"Spike at: {np.round(t, 1)} ms")

print(f"\nNumber of spikes: {len(tspike_aEIF)}")

In [ ]:
for t in tspike_aEIF:
  n = int(t/dt)
  V_aEIF[n] = 20

plt.figure(figsize=(8, 6))
plt.plot(t_vector, V_aEIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('AdEx model')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(t_vector, w)
plt.xlabel('time (ms)')
plt.ylabel('w (pA)')
plt.title('Adaptation current')
plt.show()

---
## Example 5: Conductance-Based Adaptive Exponential Integrate-and-Fire (CAdEx) Model

\begin{align}
C_m \frac{dV}{dt} &= -g_L (V-E_L) + g_L \Delta_T \text{exp} \left( \frac{V-V_T}{\Delta_T} \right) - g_A(V-E_A) + I_e \\
\tau_A \frac{g_A}{dt} &= \frac{\bar{g}_A}{1+\text{exp}(\frac{V_A-V}{\Delta_A})} - g_A
\end{align}

- $C_m$ is the membrane capacitance and $g_L$ and $E_L$ are the leak conductance and reversal potential, respectively.
- $V_T$ is the spike threshold and $\Delta_T$ is the slope of the spike initiation, $E_A$ is the reversal potential of the adaptation conductance, and $I_e$ is an input current.
- $\tau_A$ is the time constant of adaptation, $\bar{g}_A$ is the maximal subthreshold adaptation conductance, $V_A$ is the subthreshold adaptation activation voltage, and $\Delta_A$ is the slope of subthreshold adaptation.

Spike mechanism:

\begin{equation}
\text{if } V \ge V_{peak} \text{ then } V → V_r, g_A → g_A + \delta g_A
\end{equation}

In [ ]:
# Simulation parameters
T = 2000  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

# Model Parameters
C = 300  # membrane capacitance in pF
gL = 15  # leaky conductance in nS
EL = -70  # leaky reversal potential in mV

VT = -54  # threshold in mV
DeltaT = 2  # AP sharpness (slope factor) in mV
Vr = -80  # reset in mV
Vpeak = -40 # peak potential (detection limit) in mV

# Adaptation
tau_A = 144 # adaptation time constant in ms
EA = -70  # reversal potential of the adaptation in mV
gAbar = 10  # maximal subthreshold adaptation in nS
VA = -50  # subthreshold adaptation activation voltage in mV
DeltaA = 5  # slope of subthreshold adaptation in mV
deltagA = 5 # spike-triggered adaptation in pA

In [ ]:
V_CAdEx = np.zeros((len(t_vector), ))
V_CAdEx[0] = EL
gA = np.zeros((len(t_vector), ))
gA[0] = 0
tspike = [-1]
flag = True

for n in range(steps):
  ##################################
  # YOUR CODE GOES HERE
  ##################################
  deriv = ...
  V_CAdEx[n+1] = ...

  deriv_gA = ...
  gA[n+1] = ...

  if V_CAdEx[n+1] > Vpeak and flag:
    tspike.append(n*dt)
    V_CAdEx[n+1] = Vr
    gA[n+1] += deltagA
    flag = False

  if not flag:
    V_CAdEx[n+1] = Vr

  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Print the spiketimes
tspike_CAdEx = tspike[1:]  # remove the 'fake' spiketime
for t in tspike_CAdEx:
  print(f"Spike at: {np.round(t, 1)} ms")

print(f"\nNumber of spikes: {len(tspike_CAdEx)}")

In [ ]:
for t in tspike_CAdEx:
  n = int(t/dt)
  V_CAdEx[n] = 20

plt.figure(figsize=(8, 6))
plt.plot(t_vector, V_CAdEx)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('CAdEx model')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(t_vector, gA)
plt.xlabel('time (ms)')
plt.ylabel('gA (nS)')
plt.title('Adaptation')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(t_vector, gA*(V_CAdEx-EA))
plt.xlabel('time (ms)')
plt.ylabel('IA (pA)')
plt.title('Adaptative current')
plt.show()

In [ ]:
# Plot all together
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(t_vector, V_LIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('LIF model')

plt.subplot(2, 2, 2)
plt.plot(t_vector, V_EIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('EIF model')

plt.subplot(2, 2, 3)
plt.plot(t_vector, V_aEIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('aEIF model')

plt.subplot(2, 2, 4)
plt.plot(t_vector, V_CAdEx)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.title('CAdEx model')

plt.tight_layout()
plt.show()

---
## How to add a synapse model using differential equations - Exponential synapse

Let's assume that our synapse model is the simple exponential decay. We know that:

\begin{equation}
g_{syn}(t) = \bar{g}_{syn} \text{exp} \left( -\frac{t-t_s}{\tau} \right) \mathcal{H}(t-t_s)
\end{equation}

This is translated into a differential equation:

\begin{equation}
\tau \frac{dg_{syn}}{dt} = -g_{syn} + \delta(t-t_s)
\end{equation}

where $\delta(\cdot)$ is the Dirac function:


\begin{equation}
  \delta(t-\theta) = \cases{1 & $\text{if } t = \theta$ \cr
                          0 & \text{otherwise}
                   }
\end{equation}

and denotes the time of the presynaptic spike.

In [ ]:
# Simulation parameters
T = 500  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

V_LIF = np.zeros((len(t_vector), ))
V_LIF[0] = EL
tspike = [-1]  # add a negative value for programming convinience.
flag = True

g_syn = np.zeros((len(t_vector), ))
I_syn = np.zeros((len(t_vector), ))
tau = 10  # ms
Esyn = 0  # mV
gbar = 0.2  # nS

# generate random presynaptic spikes
nspikes_pre = 10
pre = np.random.choice(t_vector, size=nspikes_pre, replace=False)

for n in range(steps):
  # Check if there is a presynaptic spike
  if n*dt in pre:
    g_syn[n] += 1

  ##################################
  # YOUR CODE GOES HERE
  ##################################
  # Calculate the synaptic current
  I_syn[n] = ...

  # Calculate the derivatives of the equations
  deriv = (-gL*(V_LIF[n]-EL) + I_syn[n])/Cm
  deriv_g = -g_syn[n]/tau
  # Update using the Euler method
  V_LIF[n+1] = V_LIF[n] + dt*deriv
  g_syn[n+1] = g_syn[n] + dt*deriv_g

  # Spiking mechanism
  if V_LIF[n+1] > Vth and flag:
    tspike.append(n*dt)
    V_LIF[n+1] = Vr
    flag = False

  # Clamp voltage at Vr
  if not flag:
    V_LIF[n + 1] = Vr

  # Change the flag of refractory period
  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Draw nice spikes
for t in tspike[1:]:
  n = int(t/dt)
  V_LIF[n] = 20

# Plot of the membrane potential and synaptic conductance
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(t_vector, V_LIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')

plt.subplot(1, 2, 2)
plt.plot(t_vector, g_syn)
plt.xlabel('time (ms)')
plt.ylabel(r'$g_{syn}$ (nS)')

plt.tight_layout()
plt.show()

---
## (Optional) How to add a synapse model using differential equations - Double exponential synapse

Let's assume that our synapse model is dual exponential with $\tau_d > \tau_r$. We know that:

\begin{equation}
g_{syn}(t) = \bar{g}_{syn} f \left( \text{exp} \left( -\frac{t-t_s}{\tau_d} \right) - \text{exp} \left( -\frac{t-t_s}{\tau_r} \right)\right) \mathcal{H}(t-t_s)
\end{equation}

where (assuming $t_s=0$)

\begin{equation}
f = \frac{1}{\left( \text{exp} \left( -\frac{t_{peak}}{\tau_d} \right) - \text{exp} \left( -\frac{t_{peak}}{\tau_r} \right)\right) \left( \frac{\tau_d \tau_r}{\tau_d - \tau_r}\right)}
\end{equation}

and

\begin{equation}
t_{peak} = \frac{\tau_d \tau_r}{\tau_d - \tau_r} \text{ln} \left( \frac{\tau_d}{\tau_r}\right)
\end{equation}


This is translated into a system of differential equations:

\begin{align}
\frac{dg_{syn}}{dt} &= -\frac{g_{syn}}{\tau_d} + x_{syn}\\
\frac{dx_{syn}}{dt} &= -\frac{x_{syn}}{\tau_r} + \delta(t-t_s)
\end{align}

where $\delta(\cdot)$ is the Dirac function:


\begin{equation}
  \delta(t-\theta) = \cases{1 & $\text{if } t = \theta$ \cr
                          0 & \text{otherwise}
                   }
\end{equation}

and denotes the time of the presynaptic spike.

<br>

**Note:**
- If $\tau_d-\tau_r$ is very small compared to $\tau_r$, this is an alpha synapse with time constant $\tau_d$, e.g., let $\tau_r=9.9 \text{ ms}$ and $\tau_d=10 \text{ ms}$.
- If $\tau_r/\tau_d$ is very small, this is a single exponential decay with time constant $\tau_d$, e.g., let $\tau_r=0.1 \text{ ms}$.

In [ ]:
# Simulation parameters
T = 500  # ms
dt = 0.1  # ms
steps = int(T/dt)
t_vector = np.linspace(0, T, steps+1)

# Model Parameters
Cm = 300  # membrane capacitance in pF
gL = 15  # leaky conductance in nS
EL = -70  # leaky reversal potential in mV
Vth = -54  # threshold in mV
Vr = -80  # reset in mV

V_LIF = np.zeros((len(t_vector), ))
V_LIF[0] = EL
tspike = [-1]  # add a negative value for programming convinience.
flag = True

g_syn = np.zeros((len(t_vector), ))
x_syn = np.zeros((len(t_vector), ))
I_syn = np.zeros((len(t_vector), ))
tau_r = 9.9  # ms
tau_d = 10  # ms
Esyn = 0  # mV
gbar = 0.2  # nS

# Calculate the theoretical peak value - scaling factor `f`
t_peak = (tau_d * tau_r) / (tau_d - tau_r) * np.log(tau_d/tau_r)
f = 1/((np.exp(-t_peak/tau_d) - np.exp(-t_peak/tau_r)) * ((tau_d * tau_r) / (tau_d - tau_r)))

# generate random presynaptic spikes
nspikes_pre = 10
pre = np.random.choice(t_vector, size=nspikes_pre, replace=False)

for n in range(steps):
  # Check if there is a presynaptic spike, if yes update x_syn
  if n*dt in pre:
    x_syn[n] += 1

  # Calculate the synaptic current
  I_syn[n] = -gbar*g_syn[n]*f*(V[n] - Esyn)
  # Calculate the derivatives of the equations
  deriv = (-gL*(V_LIF[n]-EL) + I_syn[n])/Cm
  deriv_g = -g_syn[n]/tau_d + x_syn[n]
  deriv_x = -x_syn[n]/tau_r
  # Update using the Euler method
  V_LIF[n+1] = V_LIF[n] + dt*deriv
  g_syn[n+1] = g_syn[n] + dt*deriv_g
  x_syn[n+1] = x_syn[n] + dt*deriv_x

  # Spiking mechanism
  if V_LIF[n+1] > Vth and flag:
    tspike.append(n*dt)
    V_LIF[n+1] = Vr
    flag = False

  # Clamp voltage at Vr
  if not flag:
    V_LIF[n + 1] = Vr

  # Change the flag of refractory period
  if n*dt > tref + tspike[-1]:
    flag = True

In [ ]:
# Draw nice spikes
for t in tspike[1:]:
  n = int(t/dt)
  V_LIF[n] = 20

# Plot of the membrane potential and synaptic conductance
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.plot(t_vector, V_LIF)
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')

plt.subplot(2, 2, 2)
plt.plot(t_vector, gbar*f*g_syn)
plt.xlabel('time (ms)')
plt.ylabel(r'$g_{tot}$ (nS)')

plt.subplot(2, 2, 3)
plt.plot(t_vector, g_syn)
plt.xlabel('time (ms)')
plt.ylabel(r'$g_{syn}$ (nS)')

plt.subplot(2, 2, 4)
plt.plot(t_vector, x_syn)
plt.xlabel('time (ms)')
plt.ylabel(r'$x_{syn}$ (nS)')

plt.tight_layout()
plt.show()